# [0] Imports

In [192]:
import Pkg;

using LinearAlgebra, Random, Gurobi, JuMP, Distributions, Plots, LazySets, Dates

# [1] Setup

## [1.1] Initialize variables
Many parameters need to be set up and initialized so they can be called upon later in the code according to the Overleaf document.

In [193]:
n_jobs = 25
n_vehicles_jobs = n_jobs
n_vehicles_coverage = 10
T = 2 * n_jobs
min_duration = 2
max_duration = 6
speed = 1000/6
coverage_distance = 50
size = 500
step = 50;

## [1.2] Create locations, time-windows, work load, and distances

In [194]:
# We set the seed so that we always have the same result
Random.seed!(1234)

function create_cluster_sizes(jobs_total)
    Random.seed!(1234)
    jobs_created = 0
    cluster = []
    while jobs_created != jobs_total
        num_to_add = rand(min(3, jobs_total - jobs_created) : 
            min(Int((n_jobs/5)÷1), jobs_total - jobs_created))
        jobs_created += num_to_add
        push!(cluster, num_to_add)
    end
    
    return cluster
end;

In [195]:
# Create job and depot locations, time windows and work load for each job.
time_windows = []
locations = rand(Uniform(0,size), 1, 2)
work_load = []

cluster = create_cluster_sizes(n_jobs);

In [196]:
function create_time_windows_and_work_load(cluster_sizes, locations)
    Random.seed!(1234)
    locations = rand(Uniform(0,size), 1, 2)
    for size_c in cluster
        first = rand(Uniform(0,size), 1, 2)
        locations = vcat(locations, first)

        job_begins = rand(2:10)
        job_finish = rand((job_begins+min_duration):(job_begins+max_duration))
        push!(time_windows, [job_begins, job_finish])

        time_work = rand(min_duration:max(min_duration, job_finish - job_begins))
        push!(work_load, time_work)

        for neighbour in 1:(size_c-1)
            new_x = rand(Uniform(max(0,first[1]-20), min(first[1]+20, size)), 1, 1)
            new_y = rand(Uniform(max(0,first[2]-20), min(first[2]+20, size)), 1, 1)
            new = hcat(new_x, new_y)
            locations = vcat(locations, new)

            job_begins = rand(job_finish:min(T-min_duration-2, job_finish + 6))
            job_finish = rand((job_begins+min_duration):(min(job_begins+max_duration, T-2)))
            push!(time_windows, [job_begins, job_finish])

            time_work = rand(min_duration:min(max_duration, job_finish-job_begins))
            push!(work_load, time_work)
        end
    end
    
    return [time_windows, work_load, locations]
end;

In [197]:
time_windows, work_load, locations = create_time_windows_and_work_load(cluster, locations)

distances = [LinearAlgebra.norm(locations[i, :] .- locations[j, :]) for i=1:n_jobs+1, j = 1:n_jobs+1];

# [2] Setup for Set Partitioning without CG

## [2.1] Label-Setting Algorithm

The label-setting algorithm here generates ALL feasible paths.

In [198]:
function label_setting(n, travel_time, travel_distance, windows, load)
    N = [[1]] 
    T = [ [1] ] 
    C = [0.0]  
    
    L = [1]    
    
    current_state = 1
    total_state = 1
    
    while 1==1
        
        # STEP 1: Check whether we want to move on to checking the next path
        if (L[current_state] == n+2)
            current_state += 1
            if current_state > total_state
                break
            else
                continue
            end
        end
        
        # STEP 2: Enter the for loop.
        for i in 2:(n+2)
            
            # STEP 2-1: Check that the current node is not already inside our path.
            if ~(i in N[current_state])
                
                #= STEP 2-2: Feasibility check. We must have enough time
                to go from our current location (end time T[current_state]), plus
                the travel time from that end node to the new node (travel_time[L[current_state], i]),
                plus the amount of time it takes to work the OLD job (load[L[current_state]-1]),
                plus the amount of time it takes to work the NEW job (load[i-1])
                
                this must be smaller than or equal to the end time of the window for the NEW job (windows[i-1][2])
                =#
                if L[current_state] != 1
                    cur_time = last(T[current_state])
                    dist_nec = travel_time[L[current_state], i]
                    old_job_time_nec = load[L[current_state]-1] #out of our parameters this one has no depot pad
                    new_job_time_nec = load[i-1]
                    new_window_close = windows[i-1][2]
                    
                    #if invalid, don't bother
                    if cur_time + dist_nec + old_job_time_nec + new_job_time_nec > new_window_close
                        continue
                    end
                end
                
                push!(N, copy(N[current_state]))
                push!(N[total_state+1], i)

                cur_time = last(T[current_state])
                dist_nec = travel_time[L[current_state], i]
                old_job_time = L[current_state] > 1 ? load[L[current_state]-1] : 0
                new_window_start = windows[i-1][1]
                new_times = copy(T[current_state])
                push!(new_times, max(cur_time + dist_nec + old_job_time, new_window_start))
                push!(T, new_times)

                push!(C, C[current_state] + travel_distance[L[current_state], i])

                push!(L, i)
                
                total_state += 1
        
            end
        end
        
        current_state += 1 # concluded iteration for that state in mind. Now next one
        
        #= It's possible that we've just caught up to the final total state, meaning we didn't generate
        anything and can stop here. =#
        if current_state == total_state
            break
        end     
    end

    return N, C, T
end

label_setting (generic function with 2 methods)

## [2.2] Variable Setup

We need to initialize the following:

- ALL the routes which we have, the set $\mathcal{Q}$ (done from LSA above)
- the parameter $\delta_{it}^q$ telling us if route $q$ is at $i$ at time $t$
- the parameter $u_i^q$ telling us if route $q$ visits $i$ at any time
- Route costs, $C^q$.

### [2.2.1] LSA for Superset Routes

Initializing all routes uses the LSA above.

In [199]:
distances_label = deepcopy(distances);
distances_label = hcat(distances_label, distances_label[:, 1]);
distances_label = vcat(distances_label, collect(push!(distances_label[:, 1], 0)'));

In [200]:
travel_times = ceil.(distances_label / speed);

In [201]:
windows_label = deepcopy(time_windows)
push!(windows_label, [0, 100]);

In [202]:
load_label = deepcopy(work_load)
push!(load_label, 0);

In [203]:
routes, cost, times = label_setting(n_jobs, travel_times, distances_label, windows_label, load_label);

### [2.2.2] Obtain Routes, Costs, and the associated Times

In [204]:
routes_keep = []
C_routes = []
T_routes = []

# only if it ends in the depot is it useful
for r in 1:length(routes)
    if (last(routes[r]) == n_jobs+2)
        push!(routes_keep, routes[r] .- 1)
        push!(C_routes, cost[r])
        push!(T_routes, times[r])
    end
end

In [205]:
n_routes = length(routes_keep);

### [2.2.3] Preprocess Routes to Better Format

In [206]:
function route_time_combo(route, times)
    #=
    Note that this function is slightly different than from the one in colgen!
    The route will already have been modified so as to delete the padding.
    Specifically, the depot will be at 0, not 1.
    =#
    route_info = []
    for locindex in 1:length(route)-1
        #=
        Strategy:
        If you are not at a job location, you don't need to capture
        stationary information. Otherwise you do.

        Anyways we want data moving from one to the next.
        =#
        cur_loc = route[locindex]
        cur_time = times[locindex]
        time_to_start_moving = times[locindex]

        #stationary data comes first.
        if 1 < locindex < n_jobs + 2
            cur_job_num = cur_loc

            for time in cur_time:cur_time+work_load[cur_job_num]-1
                route_detail = [ [cur_loc, time], [cur_loc, time+1], 0]
                push!(route_info, route_detail)
            end

            time_to_start_moving += work_load[cur_job_num]
        end

        new_loc = route[locindex+1]
        #then if we move, we move.
        route_detail = [ [cur_loc, time_to_start_moving], [new_loc, times[locindex+1]], distances_label[cur_loc+1, new_loc+1]]
        push!(route_info, route_detail)
    end

    return route_info
end;

In [207]:
route_time_keep = [route_time_combo(routes_keep[i], T_routes[i]) for i in 1:n_routes];

### [2.2.3] Compute $u_{i}^q$

In [208]:
function compute_u(routes)
    Q = length(routes)
    u = [[0 for q in 1:Q] for i in 1:n_jobs]
    
    for rindex in 1:Q
        route = routes[rindex]
        for arc in route
            loc1 = arc[1][1]
            loc2 = arc[2][1]
            if (1 <= loc1 <= n_jobs)
                u[loc1][rindex] = 1
            end
            if (1 <= loc2 <= n_jobs)
                u[loc2][rindex] = 1
            end
        end
    end
    return u
end
;             

In [209]:
u = compute_u(route_time_keep);

### [2.2.4] Compute $\delta_{it}^q$

In [210]:
function compute_delta(routes)
    Q = length(routes)
    delta = [[[0.0 for q in 1:Q] for t in 1:T] for i in 1:n_jobs]
    for rindex in 1:Q
        route = routes[rindex]
        for arc in route
            loc1, time1 = arc[1]
            loc2, time2 = arc[2]
            if (loc1 != 0) & (loc1 != n_jobs + 1)
                delta[loc1][time1][rindex] = 1
            end
            
            if (loc2 != 0) & (loc2 != n_jobs + 1)
                delta[loc2][time2][rindex] = 1
            end
        end
    end
    return delta
end;   

In [211]:
delta = compute_delta(route_time_keep);

# [3] SP Model

In [212]:
model_set = Model(Gurobi.Optimizer);
#model = Model(with_optimizer(Gurobi.Optimizer, TimeLimit=100));

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04


## [3.1] Decision Variables

Actually there's 1. That's $z_q^k$!

In [213]:
@variable(model_set, z[1:n_vehicles_jobs, 1:n_routes], Bin);

## [3.2] Objective

In [214]:
@objective(model_set, 
    Min, 
    sum(sum(z[k,q]*C_routes[q] for q in 1:n_routes) for k in 1:n_vehicles_jobs))
;

## [3.3] Constraints

### Each job is only done once

Look at JOB vehicles. Summing up over all routes and drivers, only one driver should use the route, and that route passes by that job, so it's a sum over $u_i^q z_q^k$.

In [215]:
@constraint(model_set, 
    unique[i in 1:n_jobs],
    sum(sum(z[k,q]*u[i][q] for q in 1:n_routes) for k in 1:n_vehicles_jobs) == 1);


### Each driver is assigned to exactly one route

For each JOB vehicle: 1 route out of all within $\mathcal{Q}$ is used.

In [216]:
@constraint(model_set, 
    driver[k in 1:n_vehicles_jobs], 
    sum(z[k,q] for q in 1:n_routes) == 1);

In [217]:
optimize!(model_set)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 50 rows, 207200 columns and 1154050 nonzeros
Model fingerprint: 0xaa18f221
Variable types: 0 continuous, 207200 integer (207200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 0 rows and 25 columns
Presolve time: 0.98s
Presolved: 50 rows, 207175 columns, 1154025 nonzeros
Variable types: 0 continuous, 207175 integer (207175 binary)
Found heuristic solution: objective 9294.0984753

Starting sifting (using dual simplex for sub-problems)...

    Iter     Pivots    Primal Obj      Dual Obj        Time
       0          0     infinity      0.0000000e+00      1s
       1         58   1.1002105e+07   1.1557212e+03      1s
       2        118   5.0024082e+06   1.0502983e+03      2s
       3        163   2

# [4] Extract Results

In [219]:
z_val = value.(z);

In [221]:
for vehicle_num in 1:n_vehicles_jobs
    for route_index in 1:n_routes
        if z_val[vehicle_num, route_index] > 0.99 && route_index > 1
            println(routes_keep[route_index])
        end
    end
end

[0, 20, 21, 22, 23, 26]
[0, 1, 2, 3, 26]
[0, 16, 17, 18, 19, 13, 14, 26]
[0, 11, 12, 15, 26]
[0, 4, 5, 6, 7, 26]
[0, 8, 9, 10, 26]
[0, 24, 25, 26]
